In [ ]:
from spectral.io import envi
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D
import os, sys

import warnings
warnings.filterwarnings('ignore')

from SC_OE import nll_fn, kernel, posterior, CorrelatedErrorModel
from utils import cos_sim
import time

### Load data

In [ ]:
# Load data from same retrieval type (SCOE) across noise levels
instrument_tag = 'ang20210710t100946'
sim_tag = 'simplain_full_raw'
exp_tag = 'indiawater_gaussaod0.2'
retrieval_tag_list = ['_v5b_uncEmp_40','_noise1.0inst_v5b_uncEmp_40','_noise2.0inst_v5b_uncEmp_40','_noise5.0inst_v5b_uncEmp_40','_noise10.0inst_v5b_uncEmp_40']
retrieval_base_list = ['scoe_retrievals_noerr' for r in retrieval_tag_list]

state_tag_list = ['state_fs' for r in retrieval_base_list]
rfl_tag_list = ['refl_fs' for r in retrieval_base_list]
fixed_state_tag_list = ['fixed_state' for r in retrieval_base_list]
uncert_atm_tag_list = ['post_uncert_fs' for r in retrieval_base_list] 
uncert_tag_list = ['post_uncert_fs' for r in retrieval_base_list] 
title_base_list = ['scoe, noerr, seg 40','scoe, noerr, seg 40, noise 1x','scoe, noerr, seg 40, noise 2x','scoe, noerr, seg 40, noise 5x','scoe, noerr, seg 40, noise 10x']

base_dir = '.'
retrieval_dir_list = [base_dir + 'retrievals/{}/{}_{}_{}{}/'.format(retrieval_base_list[ii],instrument_tag,sim_tag,exp_tag,retrieval_tag_list[ii]) for ii in range(len(retrieval_base_list))] 
data_dir = base_dir + 'data/{}_{}_{}/'.format(instrument_tag,sim_tag,exp_tag)

In [ ]:
# Load data across retrieval type for one noise level
instrument_tag = 'ang20210710t100946'
sim_tag = 'simlowaod_full_raw'
exp_tag = 'indiawater_gaussaod0.05_noise1.0inst'
retrieval_tag_list = ['','_v5b_uncEmp_40','_v5b_uncEmp_40']
retrieval_base_list = ['classic_retrievals','scoe_retrievals_noerr','scoe_retrievals_noerr']

state_tag_list = ['state','state_fs','subs_state']
rfl_tag_list = ['rfl','refl_fs','rfl']
fixed_state_tag_list = ['state','fixed_state','subs_state']
uncert_atm_tag_list = ['uncert','post_uncert_fs','subs_uncert'] 
uncert_tag_list = ['uncert','post_uncert_fs','subs_uncert'] 
title_base_list = ['classic, noise 1x','scoe, noerr, seg 40','emp line, seg 40']

base_dir = '.'
retrieval_dir_list = [base_dir + 'retrievals/{}/{}_{}_{}{}/'.format(retrieval_base_list[ii],instrument_tag,sim_tag,exp_tag,retrieval_tag_list[ii]) for ii in range(len(retrieval_base_list))] 
data_dir = base_dir + 'data/{}_{}_{}/'.format(instrument_tag,sim_tag,exp_tag)

In [ ]:
# Load wavelength and trim bands
wl = np.genfromtxt(base_dir + 'data/wl.txt')
good_bands = np.loadtxt(base_dir + 'data/good_bands.txt').astype(np.int8)
wl_nan = wl.copy()
wl_nan[np.logical_not(good_bands)] = np.nan

In [ ]:
# Function to return full-size image from empirical line segments
def get_segmented_img(data, segments):
    ret_sz = (segments.shape[0],segments.shape[1],data.shape[2]) if len(data.shape)>2 else segments.shape
    return np.reshape(np.squeeze(data)[segments.flatten()],ret_sz)

In [ ]:
# Load true state and simulated radiance
print('Loading {}...'.format(data_dir +'{}_{}_{}_simulated_rdn.hdr'.format(instrument_tag,sim_tag,exp_tag)))
synth_data = envi.open(data_dir +'{}_{}_{}_simulated_rdn.hdr'.format(instrument_tag,sim_tag,exp_tag)).open_memmap(interleave='bip')
print('Loading {}...'.format(data_dir+ '{}_{}_{}_true_state.hdr'.format(instrument_tag,sim_tag,exp_tag)))
true_state = envi.open(data_dir+ '{}_{}_{}_true_state.hdr'.format(instrument_tag,sim_tag,exp_tag)).open_memmap(interleave='bip')

In [ ]:
# Load retrieved values 
for ii in range(len(retrieval_dir_list)):
    print('Loading {}...'.format(retrieval_dir_list[ii] + 'output/{}_{}.hdr'.format(instrument_tag,state_tag_list[ii])))
state_var_list = [envi.open(retrieval_dir_list[ii] + 'output/{}_{}.hdr'.format(instrument_tag,state_tag_list[ii])).open_memmap(interleave='bip')[...,-2:] for ii in range(len(retrieval_dir_list))]
rfl_var_list = [envi.open(retrieval_dir_list[ii] + 'output/{}_{}.hdr'.format(instrument_tag,rfl_tag_list[ii])).open_memmap(interleave='bip') for ii in range(len(retrieval_dir_list))]

In [ ]:
#Load fixed state file
for ii in range(len(retrieval_dir_list)):
    print('Loading {}...'.format(retrieval_dir_list[ii] + 'output/{}_{}.hdr'.format(instrument_tag,fixed_state_tag_list[ii])))
fixed_state_var_list = [envi.open(retrieval_dir_list[ii] + 'output/{}_{}.hdr'.format(instrument_tag,fixed_state_tag_list[ii])).open_memmap(interleave='bip')[...,-2:] for ii in range(len(retrieval_dir_list))]

In [ ]:
#Load uncertainty file
for ii in range(len(retrieval_dir_list)):
    print('Loading {}...'.format(retrieval_dir_list[ii] + 'output/{}_uncert.hdr'.format(instrument_tag)))
uncert_var_list = [envi.open(retrieval_dir_list[ii] + 'output/{}_{}.hdr'.format(instrument_tag,uncert_tag_list[ii])).open_memmap(interleave='bip')[...,:425] for ii in range(len(retrieval_dir_list))]
uncert_atm_var_list = [envi.open(retrieval_dir_list[ii] + 'output/{}_{}.hdr'.format(instrument_tag,uncert_atm_tag_list[ii])).open_memmap(interleave='bip')[...,-2:] for ii in range(len(retrieval_dir_list))]

In [ ]:
#Empirical line 
empline_idx = 2
segments = np.squeeze(envi.open(retrieval_dir_list[empline_idx] + '/output/{}_lbl.hdr'.format(instrument_tag)).open_memmap(interleave='bip').astype(int))
state_var_list[empline_idx] = get_segmented_img(state_var_list[empline_idx], segments)
uncert_atm_var_list[empline_idx] = get_segmented_img(uncert_atm_var_list[empline_idx], segments)

### Plot data

In [ ]:
%matplotlib widget

In [ ]:
# Reflectance & state variable retrievals
plt_chan = 100
ylabel_list = [f'Reflectance, ch {plt_chan}', 'Water Vapor','AOD']
title_list = title_base_list.copy()
title_list.append('true state')

fig = plt.figure(figsize=(10,8),constrained_layout = True)
gs = fig.add_gridspec(ncols=len(title_list), nrows=len(ylabel_list))

min_true = np.min(true_state[:,:,[plt_chan,426,425]],axis=(0,1))
max_true = np.max(true_state[:,:,[plt_chan,426,425]],axis=(0,1))

clim_values = np.zeros((len(ylabel_list),2))
for jj in range(len(ylabel_list)):
    if jj==0:
        clim_values[jj,0] = np.minimum(np.min(rfl_var_list[1][:,:,plt_chan]),min_true[jj]) 
        clim_values[jj,1] = np.maximum(np.max(rfl_var_list[1][:,:,plt_chan]),max_true[jj])
    else:
        clim_values[jj,0] = np.minimum(np.min(state_var_list[1][:,:,-jj]),min_true[jj]) 
        clim_values[jj,1] = np.maximum(np.max(state_var_list[1][:,:,-jj]),max_true[jj])
            
for ii in range(len(retrieval_dir_list)):
    for jj in range(len(ylabel_list)):
        ax = plt.subplot(gs[jj,ii])
        if jj==0:
            plt.title(title_list[ii])
            plt.imshow(rfl_var_list[ii][:,:,plt_chan]); plt.xticks([]); plt.yticks([])
        else:
            plt.imshow(state_var_list[ii][:,:,-(jj)]); plt.xticks([]); plt.yticks([])
        plt.colorbar(ax=ax,shrink=0.5)
        plt.clim(clim_values[jj,:])
        if ii == 0:
            plt.ylabel(ylabel_list[jj])
            
ii = len(retrieval_dir_list)
for jj in range(len(ylabel_list)):
    ax = plt.subplot(gs[jj,ii])
    if jj==0:
        plt.title(title_list[ii])
        plt.imshow(true_state[:,:,plt_chan]); plt.xticks([]); plt.yticks([])
    else:
        plt.imshow(true_state[:,:,-(jj)]); plt.xticks([]); plt.yticks([])
    plt.colorbar(ax=ax,shrink=0.5)
    plt.clim(clim_values[jj,:])

In [ ]:
plt.close('all')

In [ ]:
# Calculate error
good_bands_idx = good_bands.astype(bool)
stop_row = rfl_var_list[0].shape[0]
n_files = len(retrieval_dir_list)
rmse_rfl_pixel = np.zeros((n_files,stop_row,rfl_var_list[0].shape[1]))
rmse_rfl_wavelength = np.zeros((n_files,np.sum(good_bands)))
rmse_rfl = np.zeros(n_files)

abse_state_pixel = np.zeros((n_files,stop_row,rfl_var_list[0].shape[1],2))
rmse_state = np.zeros((n_files,2))

for ii in range(len(retrieval_dir_list)):
    rmse_rfl_pixel[ii,:,:] = np.sqrt(np.mean((rfl_var_list[ii][:stop_row,:,:425][...,good_bands_idx]-true_state[:stop_row,:,:-2][...,good_bands_idx])**2,axis=2))
    rmse_rfl_wavelength[ii,:] = np.sqrt(np.mean((rfl_var_list[ii][:stop_row,:,:425][...,good_bands_idx]-true_state[:stop_row,:,:-2][...,good_bands_idx])**2,axis=(0,1)))
    rmse_rfl[ii] = np.sqrt(np.mean((rfl_var_list[ii][:stop_row,:,:425][...,good_bands_idx]-true_state[:stop_row,:,:-2][...,good_bands_idx])**2))
    
    abse_state_pixel[ii,:,:,:] = np.abs(state_var_list[ii][:stop_row,:,:]-true_state[:stop_row,:,-2:])
    rmse_state[ii,:] = np.sqrt(np.mean((state_var_list[ii][:stop_row,:,:]-true_state[:stop_row,:,-2:])**2,axis=(0,1)))

In [ ]:
# Plot spectral RMSE across runs
title_list = title_base_list.copy()
plt_idx = np.arange(len(title_list)) 
plt.figure()
plt.plot(wl[good_bands_idx],np.transpose(rmse_rfl_wavelength[plt_idx,:]))
plt.legend([title_list[idx] for idx in plt_idx])
plt.title('Reflectance RMSE across wavelength')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance RMSE')

In [ ]:
# Plot spatial error across runs
plt_chan = 100
ylabel_list = ['RMSE reflectance\n(all wavelengths)', 'Abs error\nWater Vapor','Abs Error\nAOD']
title_list = title_base_list.copy()

fig = plt.figure(figsize=(8,5.8),constrained_layout = True)
gs = fig.add_gridspec(ncols=len(retrieval_dir_list), nrows=len(ylabel_list))

clim_idx = 2
clim_values = np.zeros((len(ylabel_list),2))
for jj in range(len(ylabel_list)):
    if jj==0:
        clim_values[jj,0] = np.min(rmse_rfl_pixel[clim_idx,:,:]) #Use classic retrieval to set clim
        clim_values[jj,1] = np.max(rmse_rfl_pixel[clim_idx,:,:])
    else:
        clim_values[jj,0] = np.min(abse_state_pixel[clim_idx,:,:,-jj]) #Use classic retrieval to set clim
        clim_values[jj,1] = np.max(abse_state_pixel[clim_idx,:,:,-jj])
            
for ii in range(len(title_list)):
    for jj in range(len(ylabel_list)):
        ax = plt.subplot(gs[jj,ii])
        if jj==0:
            plt.title(title_list[ii])
            plt.imshow(np.squeeze(rmse_rfl_pixel[ii,:,:])); plt.xticks([]); plt.yticks([])
        else:
            plt.imshow(np.squeeze(abse_state_pixel[ii,:,:,-jj])); plt.xticks([]); plt.yticks([])
        plt.colorbar(ax=ax,shrink=0.5)
        plt.clim(clim_values[jj,:])
        if ii == 0:
            plt.ylabel(ylabel_list[jj])

In [ ]:
# Plot uncertainty
good_bands_idx = good_bands.astype(bool)
plt_chan = 100
ylabel_list = ['Mean uncert reflectance\n(all wavelengths)', 'Uncertainty\nWater Vapor','Uncertainty\nAOD']
#title_list = ['classic','scoe, ce','scoe, cosdis','scoe, noerr']
title_list = title_base_list.copy()

fig = plt.figure(figsize=(8,5.8),constrained_layout = True)
gs = fig.add_gridspec(ncols=len(retrieval_dir_list), nrows=len(ylabel_list))

clim_values = np.zeros((len(ylabel_list),2))
for jj in range(len(ylabel_list)):
    if jj==0:
        clim_values[jj,0] = np.min(np.mean(uncert_var_list[0][:,:,:425][:,:,good_bands_idx],axis=2)) #Use classic retrieval to set clim
        clim_values[jj,1] = np.max(np.mean(uncert_var_list[0][:,:,:425][:,:,good_bands_idx],axis=2))
    else:
        clim_values[jj,0] = np.min(uncert_atm_var_list[0][:,:,-jj]) #Use classic retrieval to set clim
        clim_values[jj,1] = np.max(uncert_atm_var_list[0][:,:,-jj])
            
for ii in range(len(title_list)):
    for jj in range(len(ylabel_list)):
        ax = plt.subplot(gs[jj,ii])
        if jj==0:
            plt.title(title_list[ii])
            plt.imshow(np.squeeze(np.mean(uncert_var_list[ii][:,:,:425][:,:,good_bands_idx],axis=2))); plt.xticks([]); plt.yticks([])
        else:
            plt.imshow(np.squeeze(uncert_atm_var_list[ii][:,:,-jj])); plt.xticks([]); plt.yticks([])
        plt.colorbar(ax=ax,shrink=0.5)
        plt.clim(clim_values[jj,:])
        if ii == 0:
            plt.ylabel(ylabel_list[jj])

In [ ]:
# Plot spectra at select points alongside image comparison

plot_chn = 100

fig = plt.figure(figsize=(12,10),constrained_layout = True)
gs = fig.add_gridspec(ncols=6, nrows=4) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

plot_x = np.array([241,240,240,241,240])+3
plot_y = np.array([66,58,44,192,6])

ax = plt.subplot(gs[0,0])
plt.imshow(rmse_rfl_pixel[0,:,:]); plt.xticks([]); plt.yticks([])
plt.ylabel('Classic, RMSE')
plt.colorbar(ax=ax,shrink=0.5)
plt.title('Reflectance')
#plt.scatter(plot_x,plot_y,marker='o')

ax = plt.subplot(gs[1,0])
plt.imshow(rfl_var_list[0][:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('Classic, Retrieval')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[2,0])
plt.imshow(true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('True')
plt.colorbar(ax=ax,shrink=0.5)


for ii in range(plot_x.shape[0]):
    ax = plt.subplot(gs[2:,1:])
    plt.plot(wl_nan,np.squeeze(true_state[plot_y[ii],plot_x[ii],:-2]))
    plt.plot(wl_nan,np.squeeze(rfl_var_list[0][plot_y[ii],plot_x[ii],:-2]),color='k',linestyle='dashed')
    plt.title('True and retrieved reflectance')
    plt.legend(['True state','Classic retrieval'])
    
    # ax = plt.subplot(gs[2:,1:])
    # plt.plot(wl_nan,np.squeeze(synth_data[plot_y[ii],plot_x[ii],:]))
    # plt.title('Simulated radiance')
    # plt.legend(['Simulated'])
    
    ax = plt.subplot(gs[:2,1:])
    plt.plot(wl_nan,np.abs(np.squeeze(rfl_var_list[0][plot_y[ii],plot_x[ii],:-2])-np.squeeze(true_state[plot_y[ii],plot_x[ii],:-2])))
    plt.title('Absolute error')
    plt.legend(['Classic'])
     
    ax = plt.subplot(gs[0,0])
    plt.scatter(plot_x[ii],plot_y[ii],marker='o')

In [ ]:
# Load all
instrument_tag = 'ang20210710t100946'
sim_tag = 'simclassic_full_raw'
exp_tag = 'indiawater_gaussaod0.2_noAz'
retrieval_tag_list = ['_v5bounds_unc0.0001_emitprior_40','_v5bounds_unc0.01_emitprior_40','_v5bounds_emitprior_40']
retrieval_base_list = ['scoe_retrievals_noerr','scoe_retrievals_noerr','scoe_retrievals_noerr']#,'scoe_retrievals_ce','scoe_retrievals_cosdis']#,'scoe_retrievals_ce','scoe_retrievals_cosdis','scoe_retrievals_noerr']

title_base_list = ['scoe, noerr, seg 40, unc 0.0001','scoe, noerr, seg 40, unc 0.01','scoe, noerr, seg 40, unc emp']#,'scoe, ce (corr error)','scoe, cosdis']

base_dir = '/beegfs/scratch/reckert/develop/SC_OE/'
retrieval_dir_list = [base_dir + 'retrievals/{}/{}_{}_{}{}/'.format(retrieval_base_list[ii],instrument_tag,sim_tag,exp_tag,retrieval_tag_list[ii]) for ii in range(len(retrieval_base_list))] 
data_dir = base_dir + 'data/{}_{}_{}/'.format(instrument_tag,sim_tag,exp_tag)

In [ ]:
# Load all
instrument_tag = 'ang20210710t100946'
sim_tag = 'simplain_full_raw'
exp_tag = 'indiawater_gaussaod0.2'
retrieval_tag = '_40'
retrieval_base_list = ['scoe_retrievals_noerr']

title_base_list = ['scoe, noerr']

base_dir = '/beegfs/scratch/reckert/develop/SC_OE/'
retrieval_dir_list = [base_dir + 'retrievals/{}/{}_{}_{}{}/'.format(retrieval_base_list[ii],instrument_tag,sim_tag,exp_tag,retrieval_tag) for ii in range(len(retrieval_base_list))] 
data_dir = base_dir + 'data/{}_{}_{}/'.format(instrument_tag,sim_tag,exp_tag)

In [ ]:
# Load all
instrument_tag = 'ang20210710t100946'
sim_tag = 'simclassic_full_raw'
exp_tag = 'indiawater_gaussaod0.2'
retrieval_tag = '_emitprior_100'
retrieval_base_list = ['scoe_retrievals_noerr','scoe_retrievals_ce','scoe_retrievals_cosdis','scoe_retrievals_cosdis_norm','scoe_retrievals_pca_cosdis','scoe_retrievals_pca_cosdis_norm']#,'scoe_retrievals_ce','scoe_retrievals_cosdis','scoe_retrievals_noerr']

title_base_list = ['scoe, noerr','scoe, ce (corr error)','scoe, cosdis', 'scoe, cosdis norm','scoe, pca cosdis','scoe, pca cosdis norm']

base_dir = '/beegfs/scratch/reckert/develop/SC_OE/'
retrieval_dir_list = [base_dir + 'retrievals/{}/{}_{}_{}{}/'.format(retrieval_base_list[ii],instrument_tag,sim_tag,exp_tag,retrieval_tag) for ii in range(len(retrieval_base_list))] 
data_dir = base_dir + 'data/{}_{}_{}/'.format(instrument_tag,sim_tag,exp_tag)

In [ ]:
# Load wavelength and trim bands
wl = np.genfromtxt(base_dir + 'data/wl.txt')
good_bands = np.loadtxt(base_dir + 'data/good_bands.txt').astype(np.int8)
wl_nan = wl.copy()
wl_nan[np.logical_not(good_bands)] = np.nan

In [ ]:
# Load true state and simulated radiance
print('Loading {}...'.format(data_dir +'{}_{}_{}_simulated_rdn.hdr'.format(instrument_tag,sim_tag,exp_tag)))
synth_data = envi.open(data_dir +'{}_{}_{}_simulated_rdn.hdr'.format(instrument_tag,sim_tag,exp_tag)).open_memmap(interleave='bip')
print('Loading {}...'.format(data_dir+ '{}_{}_{}_true_state.hdr'.format(instrument_tag,sim_tag,exp_tag)))
true_state = envi.open(data_dir+ '{}_{}_{}_true_state.hdr'.format(instrument_tag,sim_tag,exp_tag)).open_memmap(interleave='bip')

In [ ]:
#Load data
aot550_gpr_coeffs = np.zeros((len(retrieval_dir_list),2))
h2ostr_gpr_coeffs = aot550_gpr_coeffs.copy()
for ii in range(len(retrieval_dir_list)):
    print('Loading {}...'.format(retrieval_dir_list[ii] + 'output/{}_fixed_state.hdr'.format(instrument_tag)))
    aot550_gpr_coeffs[ii,] = np.loadtxt(retrieval_dir_list[ii] + 'output/AOT550_gpr_coefficients.txt')
    h2ostr_gpr_coeffs[ii,] = np.loadtxt(retrieval_dir_list[ii] + 'output/H2OSTR_gpr_coefficients.txt')
fixed_state_var_list = [envi.open(retrieval_dir_list[ii] + 'output/{}_fixed_state.hdr'.format(instrument_tag)).open_memmap(interleave='bip')[...,-2:] for ii in range(len(retrieval_dir_list))]
subs_state_var_list = [envi.open(retrieval_dir_list[ii] + 'output/{}_subs_state.hdr'.format(instrument_tag)).open_memmap(interleave='bip') for ii in range(len(retrieval_dir_list))]
lbl_var_list = [envi.open(retrieval_dir_list[ii] + 'output/{}_lbl.hdr'.format(instrument_tag)).open_memmap(interleave='bip') for ii in range(len(retrieval_dir_list))]

In [ ]:
print(aot550_gpr_coeffs)
print(h2ostr_gpr_coeffs)

In [ ]:
plt.figure()
plt.plot(aot550_gpr_coeffs)
plt.plot(h2ostr_gpr_coeffs)
plt.legend(['AOD l0','AOD sigma_f','H2O l0','H2O sigma_f'])
plt.title('GPR coefficients across error methods')

In [ ]:
%matplotlib widget

In [ ]:

title_list = ['Water Vapor','AOD']
ylabel_list = title_base_list.copy() 
ylabel_list.append('true state')

fig = plt.figure(figsize=(8,8),constrained_layout = True)
gs = fig.add_gridspec(ncols=len(title_list), nrows=len(ylabel_list))

clim_values = np.zeros((len(title_list),2))
for jj in range(len(title_list)):
    clim_values[jj,0] = np.min(fixed_state_var_list[0][:,:,-(jj+1)]) #Use SCOE, noerr to set clim
    clim_values[jj,1] = np.max(fixed_state_var_list[0][:,:,-(jj+1)])
    #clim_values[jj,0] = np.min(true_state[:,:,-(jj+1)]) #Use true state
    #clim_values[jj,1] = np.max(true_state[:,:,-(jj+1)])

for ii in range(len(ylabel_list)):
    for jj in range(len(title_list)):
        ax = plt.subplot(gs[ii,jj])
        if ii == (len(ylabel_list)-1):
            plt.imshow(true_state[:,:,-(jj+1)]); plt.xticks([]); plt.yticks([])
        else:
            plt.imshow(fixed_state_var_list[ii][:,:,-(jj+1)]); plt.xticks([]); plt.yticks([])
        plt.colorbar(ax=ax,shrink=0.5)
        if jj==0:
            plt.clim(clim_values[jj,:])
        if jj == 0:
            plt.ylabel(ylabel_list[ii])
        if ii == 0:
            plt.title(title_list[jj])

In [ ]:
plt.figure()
plt.imshow(np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(true_state[:,:,-1]))))[150:250,150:275])
plt.colorbar()
plt.clim((0,1e0))
plt.title('Fourier Transform(True State, water vapor)')

In [ ]:
plt.title('Fourier Transform(GPR, Water Vapor (original))')

In [ ]:
length_scale_b = np.array([[1e-4, 1e-1],[1e-8,1e-5]]) 

In [ ]:
length_scale_b[0,1]

In [ ]:
plt.clim((0,1e1))

In [ ]:
plt.figure()
plt.imshow(np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(fixed_state_var_list[0][:,:,-(1)]))))[150:250,150:275])
plt.colorbar()
plt.clim((0,1e0))
plt.title('Fourier Transform(GPR, Waver Vapor (original))')

In [ ]:
title_base_list

In [ ]:
plt.close('all')

In [ ]:
title_list = ['Error,\nWater Vapor','Error,\nAOD']
ylabel_list = title_base_list.copy() 

fig = plt.figure(figsize=(8,12),constrained_layout = True)
gs = fig.add_gridspec(ncols=len(title_list), nrows=len(ylabel_list))

clim_values = np.zeros((len(title_list),2))
for jj in range(len(title_list)):
    # clim_values[jj,0] = np.min(fixed_state_var_list[0][:,:,-(jj+1)]) #Use SCOE, noerr to set clim
    # clim_values[jj,1] = np.max(fixed_state_var_list[0][:,:,-(jj+1)])
    clim_values[jj,0] = np.min(fixed_state_var_list[0][:,:,-(jj+1)]-true_state[:,:,-(jj+1)]) #Use true state
    clim_values[jj,1] = np.max(fixed_state_var_list[0][:,:,-(jj+1)]-true_state[:,:,-(jj+1)])
            
for ii in range(len(ylabel_list)):
    for jj in range(len(title_list)):
        ax = plt.subplot(gs[ii,jj])
        plt.imshow(fixed_state_var_list[ii][:,:,-(jj+1)]-true_state[:,:,-(jj+1)]); plt.xticks([]); plt.yticks([])
        plt.colorbar(ax=ax,shrink=0.5)
        plt.clim(clim_values[jj,:])
        if jj == 0:
            plt.ylabel(ylabel_list[ii])
        if ii == 0:
            plt.title(title_list[jj])

In [ ]:
good_bands_idx = good_bands.astype(bool)
#stop_row = 159 #For testsim, did not run classic all the way, needed to only look at RMSE to a certain row
stop_row = fixed_state_var_list[0].shape[0]
n_files = len(retrieval_dir_list)

abse_state_pixel = np.zeros((n_files,stop_row,fixed_state_var_list[0].shape[1],2))
rmse_state = np.zeros((n_files,2))

for ii in range(len(retrieval_dir_list)):    
    abse_state_pixel[ii,:,:,:] = np.abs(fixed_state_var_list[ii][:stop_row,:,-2:]-true_state[:stop_row,:,-2:])
    rmse_state[ii,:] = np.sqrt(np.mean((fixed_state_var_list[ii][:stop_row,:,-2:]-true_state[:stop_row,:,-2:])**2,axis=(0,1)))

In [ ]:
rmse_state

In [ ]:
ylabel_list = ['Water Vapor','AOD']
title_list = title_base_list.copy() 

fig = plt.figure(figsize=(8,8),constrained_layout = True)
gs = fig.add_gridspec(ncols=len(retrieval_dir_list), nrows=len(ylabel_list))

clim_values = np.zeros((len(ylabel_list),2))
for jj in range(len(ylabel_list)):
    clim_values[jj,0] = np.min(fixed_state_var_list[1][:,:,-(jj+1)]) #Use SCOE, CE to set clim
    clim_values[jj,1] = np.max(fixed_state_var_list[1][:,:,-(jj+1)])
            
for ii in range(len(retrieval_dir_list)):
    for jj in range(len(ylabel_list)):
        ax = plt.subplot(gs[jj,ii])
        plt.imshow(fixed_state_var_list[ii][:,:,-(jj+1)]); plt.xticks([]); plt.yticks([])
        plt.colorbar(ax=ax,shrink=0.5)
        plt.clim(clim_values[jj,:])
        if ii == 0:
            plt.ylabel(ylabel_list[jj])
        if jj==0:
            plt.title(title_list[ii])

In [ ]:
%matplotlib widget

In [ ]:
instrument_tag = 'ang20210710t100946'
sim_tag = 'simplain_full_raw'
exp_tag = 'indiawater_gaussaod0.2'
retrieval_tag = '_40'

base_dir = '/beegfs/scratch/reckert/develop/SC_OE/'
retrieval_dir = base_dir + 'retrievals/empline_retrievals/{}_{}_{}{}/'.format(instrument_tag,sim_tag,exp_tag,retrieval_tag)
data_dir = base_dir + 'data/{}_{}_{}/'.format(instrument_tag,sim_tag,exp_tag)

In [ ]:
# Load wavelength and trim bands
wl = np.genfromtxt(base_dir + 'data/wl.txt')
good_bands = np.loadtxt(base_dir + 'data/good_bands.txt').astype(np.int8)
wl_nan = wl.copy()
wl_nan[np.logical_not(good_bands)] = np.nan

In [ ]:
# Load true state and simulated radiance
print('Loading {}...'.format(data_dir +'{}_{}_{}_simulated_rdn.hdr'.format(instrument_tag,sim_tag,exp_tag)))
synth_data = envi.open(data_dir +'{}_{}_{}_simulated_rdn.hdr'.format(instrument_tag,sim_tag,exp_tag)).open_memmap(interleave='bip')
# Load retrieved values (only classic, for now)
print('Loading {}...'.format(data_dir+ '{}_{}_{}_true_state.hdr'.format(instrument_tag,sim_tag,exp_tag)))
print('Loading {}...'.format(retrieval_dir + 'output/{}_refl_fs.hdr'.format(instrument_tag)))
true_state = envi.open(data_dir+ '{}_{}_{}_true_state.hdr'.format(instrument_tag,sim_tag,exp_tag)).open_memmap(interleave='bip')
inde_rfl = envi.open(retrieval_dir + 'output/{}_refl_fs.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
inde_state = envi.open(retrieval_dir + 'output/{}_state_fs.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
inde_fixed_state = envi.open(retrieval_dir + 'output/{}_fixed_state.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
inde_subs_rfl = envi.open(retrieval_dir + 'output/{}_subs_rfl.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
inde_subs_state = envi.open(retrieval_dir + 'output/{}_subs_state.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
segments = np.squeeze(envi.open(retrieval_dir + '/output/{}_lbl.hdr'.format(instrument_tag)).open_memmap(interleave='bip').astype(int))

In [ ]:
# Load true state and simulated radiance
#Empline only
print('Loading {}...'.format(data_dir +'{}_{}_{}_simulated_rdn.hdr'.format(instrument_tag,sim_tag,exp_tag)))
synth_data = envi.open(data_dir +'{}_{}_{}_simulated_rdn.hdr'.format(instrument_tag,sim_tag,exp_tag)).open_memmap(interleave='bip')
# Load retrieved values (only classic, for now)
print('Loading {}...'.format(data_dir+ '{}_{}_{}_true_state.hdr'.format(instrument_tag,sim_tag,exp_tag)))
print('Loading {}...'.format(retrieval_dir + 'output/{}_refl_fs.hdr'.format(instrument_tag)))
true_state = envi.open(data_dir+ '{}_{}_{}_true_state.hdr'.format(instrument_tag,sim_tag,exp_tag)).open_memmap(interleave='bip')
inde_rfl = envi.open(retrieval_dir + 'output/{}_rfl.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
inde_subs_rfl = envi.open(retrieval_dir + 'output/{}_subs_rfl.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
inde_subs_state = envi.open(retrieval_dir + 'output/{}_subs_state.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
segments = np.squeeze(envi.open(retrieval_dir + '/output/{}_lbl.hdr'.format(instrument_tag)).open_memmap(interleave='bip').astype(int))

In [ ]:
state_img = get_segmented_img(inde_subs_state[...,-2:],segments)

In [ ]:
inde_rfl_emp = envi.open(retrieval_dir + 'output/{}_rfl.hdr'.format(instrument_tag)).open_memmap(interleave='bip')

In [ ]:
state_img.shape

In [ ]:
def get_segmented_img(data, segments):
    ret_sz = (segments.shape[0],segments.shape[1],data.shape[2]) if len(data.shape)>2 else segments.shape
    return np.reshape(np.squeeze(data)[segments.flatten()],ret_sz)

In [ ]:
%matplotlib widget

In [ ]:
plot_chn = 100

fig = plt.figure(figsize=(12,6),constrained_layout = True)
gs = fig.add_gridspec(ncols=4, nrows=3) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

clim_rfl = (0,0.75)
clim_wv  = (1.31,1.44)
#clim_aod = (0.2, 0.25)
# True State Values

ax = plt.subplot(gs[0,0])
plt.imshow(true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.title('True State')
plt.ylabel('Reflectance, 887 nm')
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_rfl)

ax = plt.subplot(gs[1,0])
plt.imshow(true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.ylabel('Water Vapor')
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_wv)

ax = plt.subplot(gs[2,0])
im = plt.imshow(true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.ylabel('AOD')
plt.colorbar(ax=ax,shrink=0.5)

# Retrieval (subs)

ax = plt.subplot(gs[0,1])
plt.imshow(get_segmented_img(inde_subs_rfl[:,:,plot_chn],segments)); plt.xticks([]); plt.yticks([])
plt.title('Emp line, subs (segments)')
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_rfl)

ax = plt.subplot(gs[1,1])
plt.imshow(state_img[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_wv)

ax = plt.subplot(gs[2,1])
plt.imshow(state_img[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
#plt.clim(clim_aod)


# Retrieval 
ax = plt.subplot(gs[0,2])
plt.title('Emp line, full rfl')
plt.imshow(inde_rfl[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_rfl)


In [ ]:
plot_chn = 100

fig = plt.figure(figsize=(12,6),constrained_layout = True)
gs = fig.add_gridspec(ncols=4, nrows=3) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

clim_rfl = (0,0.75)
clim_wv  = (1.31,1.44)
#clim_aod = (0.2, 0.25)
# True State Values

ax = plt.subplot(gs[0,0])
plt.imshow(true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.title('True State')
plt.ylabel('Reflectance, 887 nm')
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_rfl)

ax = plt.subplot(gs[1,0])
plt.imshow(true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.ylabel('Water Vapor')
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_wv)

ax = plt.subplot(gs[2,0])
im = plt.imshow(true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.ylabel('AOD')
plt.colorbar(ax=ax,shrink=0.5)

# Retrieval (subs)

ax = plt.subplot(gs[0,1])
plt.imshow(get_segmented_img(inde_subs_rfl[:,:,plot_chn],segments)); plt.xticks([]); plt.yticks([])
plt.title('SCOE, subs (segments)')
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_rfl)

ax = plt.subplot(gs[1,1])
plt.imshow(get_segmented_img(inde_subs_state[:,:,-1],segments)); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_wv)

ax = plt.subplot(gs[2,1])
plt.imshow(get_segmented_img(inde_subs_state[:,:,-2],segments)); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
#plt.clim(clim_aod)

# Retrieval (fixed state)


ax = plt.subplot(gs[1,2])
plt.title('SCOE, GPR estimate')
plt.imshow(inde_fixed_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_wv)

ax = plt.subplot(gs[2,2])
plt.imshow(inde_fixed_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
#plt.clim(clim_aod)

# Retrieval (fixed state)
ax = plt.subplot(gs[0,3])
plt.title('SCOE, full retrieval')
plt.imshow(inde_rfl[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_rfl)

ax = plt.subplot(gs[1,3])
plt.imshow(inde_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_wv)

ax = plt.subplot(gs[2,3])
plt.imshow(inde_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
#plt.clim(clim_aod)


In [ ]:
plot_chn = 60
plt.figure()
plt.imshow(inde_rfl[:,:,plot_chn]-true_state[:,:,plot_chn])
plt.colorbar(shrink=0.5)

In [ ]:
plt.close('all')

In [ ]:

plot_chn = 100

fig = plt.figure(figsize=(8,12),constrained_layout = True)
gs = fig.add_gridspec(ncols=3, nrows=5) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

# True State Values

ax = plt.subplot(gs[0,0])
plt.imshow(true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('True')
plt.title('Reflectance')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[0,1])
plt.imshow(true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.title('Water Vapor')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[0,2])
im = plt.imshow(true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.title('AOD')
plt.colorbar(ax=ax,shrink=0.5)

# Retrieval (subs)

ax = plt.subplot(gs[1,0])
plt.imshow(get_segmented_img(inde_subs_rfl[:,:,plot_chn],segments)); plt.xticks([]); plt.yticks([])
plt.ylabel('SCOE, subs (segments)')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[1,1])
plt.imshow(get_segmented_img(inde_subs_state[:,:,-1],segments)); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[1,2])
plt.imshow(get_segmented_img(inde_subs_state[:,:,-2],segments)); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

# Retrieval (fixed state)


ax = plt.subplot(gs[2,1])
plt.ylabel('SCOE, GPR estimate')
plt.imshow(inde_fixed_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[2,2])
plt.imshow(inde_fixed_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

# Retrieval (fixed state)
ax = plt.subplot(gs[3,0])
plt.ylabel('SCOE, full retrieval')
plt.imshow(inde_rfl[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[3,1])
plt.imshow(inde_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[3,2])
plt.imshow(inde_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)


# Abs(error)/true state

ax = plt.subplot(gs[4,0])
plt.imshow(np.abs(inde_rfl[:,:,plot_chn]-true_state[:,:,plot_chn])/true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('Abs(err)/true')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[4,1])
plt.imshow(np.abs(inde_state[:,:,-1]-true_state[:,:,-1])/true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[4,2])
plt.imshow(np.abs(inde_state[:,:,-2]-true_state[:,:,-2])/true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)



In [ ]:
# Abs(error)/true state
plot_chn = 100

fig = plt.figure(figsize=(12,6),constrained_layout = True)
gs = fig.add_gridspec(ncols=4, nrows=3) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)
ax = plt.subplot(gs[0,0])
plt.ylabel('Reflectance, 887 nm')
#plt.imshow(np.abs(get_segmented_img(inde_subs_rfl[:,:,plot_chn],segments)-true_state[:,:,plot_chn])/true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.imshow(np.abs(inde_rfl_emp[:,:,plot_chn]-true_state[:,:,plot_chn])/true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.title('Abs(error)/true')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[1,0])
plt.imshow(np.abs(get_segmented_img(inde_subs_state[:,:,-1],segments)-true_state[:,:,-1])/true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.ylabel('Water Vapor')

ax = plt.subplot(gs[2,0])
plt.imshow(np.abs(get_segmented_img(inde_subs_state[:,:,-2],segments)-true_state[:,:,-2])/true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.ylabel('AOD')

ax = plt.subplot(gs[0,1])
#plt.imshow(np.abs(get_segmented_img(inde_subs_rfl[:,:,plot_chn],segments)-true_state[:,:,plot_chn])); plt.xticks([]); plt.yticks([])
plt.imshow(np.abs(inde_rfl_emp[:,:,plot_chn]-true_state[:,:,plot_chn])); plt.xticks([]); plt.yticks([])
plt.title('Abs(error)')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[1,1])
plt.imshow(np.abs(get_segmented_img(inde_subs_state[:,:,-1],segments)-true_state[:,:,-1])); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)


ax = plt.subplot(gs[2,1])
plt.imshow(np.abs(get_segmented_img(inde_subs_state[:,:,-2],segments)-true_state[:,:,-2])); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)



In [ ]:
# Abs(error)/true state
plot_chn = 100

fig = plt.figure(figsize=(5,10),constrained_layout = True)
gs = fig.add_gridspec(ncols=1, nrows=3) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)
ax = plt.subplot(gs[0,0])
plt.title('Reflectance, 887 nm')
plt.ylabel('error')
plt.xlabel('Uncertainty')
error_met = inde_rfl_emp[:,:,plot_chn]-true_state[:,:,plot_chn]
uncert_met = inde_uncert_init[:,:,plot_chn] #get_segmented_img(inde_subs_uncert[:,:,plot_chn],segments)
plt.scatter(uncert_met.flatten(),error_met.flatten())

ax = plt.subplot(gs[1,0])
plt.title('Water Vapor, super-pixels')
plt.ylabel('error')
plt.xlabel('Uncertainty')
error_met = get_segmented_img(inde_subs_state[:,:,-1],segments)-true_state[:,:,-1]
uncert_met = get_segmented_img(inde_subs_uncert[:,:,-1],segments)
plt.scatter(uncert_met.flatten(),error_met.flatten())

ax = plt.subplot(gs[2,0])
plt.title('AOD, super-pixels')
plt.ylabel('error')
plt.xlabel('Uncertainty')
error_met = get_segmented_img(inde_subs_state[:,:,-2],segments)-true_state[:,:,-2]
uncert_met = get_segmented_img(inde_subs_uncert[:,:,-2],segments)
plt.scatter(uncert_met.flatten(),error_met.flatten())

In [ ]:
ax = plt.subplot(gs[1,0])
plt.imshow(np.abs(get_segmented_img(inde_subs_state[:,:,-1],segments)-true_state[:,:,-1])/true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.ylabel('Water Vapor')

ax = plt.subplot(gs[2,0])
plt.imshow(np.abs(get_segmented_img(inde_subs_state[:,:,-2],segments)-true_state[:,:,-2])/true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.ylabel('AOD')

ax = plt.subplot(gs[0,1])
plt.imshow(np.abs(get_segmented_img(inde_subs_rfl[:,:,plot_chn],segments)-true_state[:,:,plot_chn])); plt.xticks([]); plt.yticks([])
plt.title('Abs(error)')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[1,1])
plt.imshow(np.abs(get_segmented_img(inde_subs_state[:,:,-1],segments)-true_state[:,:,-1])); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)


ax = plt.subplot(gs[2,1])
plt.imshow(np.abs(get_segmented_img(inde_subs_state[:,:,-2],segments)-true_state[:,:,-2])); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

In [ ]:
%matplotlib widget

In [ ]:
# Load uncertainty files
print('Loading {}...'.format(retrieval_dir + 'output/{}_uncert.hdr'.format(instrument_tag)))
inde_uncert_init = envi.open(retrieval_dir + 'output/{}_uncert.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
inde_subs_uncert = envi.open(retrieval_dir + 'output/{}_subs_uncert.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
inde_uncert_final = envi.open(retrieval_dir + 'output/{}_post_uncert_fs.hdr'.format(instrument_tag)).open_memmap(interleave='bip')


In [ ]:
inde_uncert_final.shape

In [ ]:
plot_chn = 100

fig = plt.figure(figsize=(9,6),constrained_layout = True)
gs = fig.add_gridspec(ncols=3, nrows=3) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

clim_rfl = (0.0005,0.009)
clim_wv = (0.01,0.03)
clim_aod = (0.05,0.3)

# True State Values

ax = plt.subplot(gs[0,0])
plt.imshow(true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.title('True State')
plt.ylabel('Reflectance, 887 nm')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[1,0])
plt.imshow(true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.ylabel('Water Vapor')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[2,0])
im = plt.imshow(true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.ylabel('AOD')
plt.colorbar(ax=ax,shrink=0.5)

# # Retrieval (subs)

# ax = plt.subplot(gs[1,0])
# plt.imshow(inde_uncert_init[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
# plt.ylabel('SCOE, uncertainty, initial')
# plt.colorbar(ax=ax,shrink=0.5)


# Retrieval (subs)
ax = plt.subplot(gs[0,1])
plt.imshow(inde_uncert_init[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
#plt.imshow(get_segmented_img(inde_subs_uncert[:,:,plot_chn],segments)); plt.xticks([]); plt.yticks([])
plt.title('SCOE, uncertainty, subs')
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_rfl)

ax = plt.subplot(gs[1,1])
plt.imshow(get_segmented_img(inde_subs_uncert[:,:,-1],segments)); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_wv)

ax = plt.subplot(gs[2,1])
plt.imshow(get_segmented_img(inde_subs_uncert[:,:,-2],segments)); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_aod)

# Retrieval (fixed state)
ax = plt.subplot(gs[0,2])
plt.title('SCOE, uncertainty, final')
plt.imshow(inde_uncert_final[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_rfl)

ax = plt.subplot(gs[1,2])
plt.imshow(inde_uncert_final[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_wv)

ax = plt.subplot(gs[2,2])
plt.imshow(inde_uncert_final[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_aod)

# # Retrieval (subs)

# ax = plt.subplot(gs[1,0])
# plt.imshow(inde_uncert_init[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
# plt.ylabel('SCOE, uncertainty, initial')
# plt.colorbar(ax=ax,shrink=0.5)


In [ ]:
plt.close('all')

In [ ]:
plot_chn = 100

fig = plt.figure(figsize=(8,12),constrained_layout = True)
gs = fig.add_gridspec(ncols=3, nrows=5) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

# True State Values

ax = plt.subplot(gs[0,0])
plt.imshow(true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('True')
plt.title('Reflectance')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[0,1])
plt.imshow(true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.title('Water Vapor')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[0,2])
im = plt.imshow(true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.title('AOD')
plt.colorbar(ax=ax,shrink=0.5)

# Retrieval (subs)

ax = plt.subplot(gs[1,0])
plt.imshow(inde_uncert_init[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('SCOE, uncertainty, initial')
plt.colorbar(ax=ax,shrink=0.5)


# Retrieval (subs)
ax = plt.subplot(gs[2,0])
plt.imshow(get_segmented_img(inde_subs_uncert[:,:,plot_chn],segments)); plt.xticks([]); plt.yticks([])
plt.ylabel('SCOE, uncertainty, subs')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[2,1])
plt.imshow(get_segmented_img(inde_subs_uncert[:,:,-1],segments)); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[2,2])
plt.imshow(get_segmented_img(inde_subs_uncert[:,:,-2],segments)); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

# Retrieval (fixed state)
ax = plt.subplot(gs[3,0])
plt.ylabel('SCOE, uncertainty, final')
plt.imshow(inde_uncert_final[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[3,1])
plt.imshow(inde_uncert_final[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[3,2])
plt.imshow(inde_uncert_final[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

# Abs(error)/true state

ax = plt.subplot(gs[4,0])
plt.imshow(np.abs(inde_rfl[:,:,plot_chn]-true_state[:,:,plot_chn])/true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('Abs(err)/true')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[4,1])
plt.imshow(np.abs(inde_state[:,:,-1]-true_state[:,:,-1])/true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[4,2])
plt.imshow(np.abs(inde_state[:,:,-2]-true_state[:,:,-2])/true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

In [ ]:
# Evaluating areas of high and low uncertainty
# uncert_base_vec = np.squeeze(inde_subs_uncert[:,:,-1])
# uncert_base_img = get_segmented_img(inde_subs_uncert[:,:,-1],segments)
uncert_base_vec = np.squeeze(np.mean(inde_subs_uncert[:,:,:-2],axis=2))
uncert_base_img = get_segmented_img(np.mean(inde_subs_uncert[:,:,:-2],axis=2),segments)

In [ ]:
plt.close('all')

In [ ]:
plt.figure()
plt.hist(uncert_base_vec,bins=20)

In [ ]:
percentile_q = 5
plt.figure()
plt.subplot(1,3,1)
plt.imshow(uncert_base_img)
plt.title('Uncertainty')
plt.subplot(1,3,2)
plt.imshow(uncert_base_img<=np.percentile(uncert_base_vec,q=percentile_q))
plt.title('Pixels, lowest uncert')
plt.subplot(1,3,3)
plt.imshow(uncert_base_img>=np.percentile(uncert_base_vec,q=(100-percentile_q)))
plt.title('Pixels, highest uncert')

In [ ]:
wv

In [ ]:
percentile_q = 5
low_vec_idx = uncert_base_vec<=np.percentile(uncert_base_vec,q=percentile_q)
low_img_idx = uncert_base_img<=np.percentile(uncert_base_vec,q=percentile_q)
high_vec_idx = uncert_base_vec>=np.percentile(uncert_base_vec,q=(100-percentile_q))
high_img_idx = uncert_base_img>=np.percentile(uncert_base_vec,q=(100-percentile_q))

fig = plt.figure(figsize=(8,10),constrained_layout = True)
gs = fig.add_gridspec(ncols=3, nrows=4) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

# Subs uncertainty, all
ax = plt.subplot(gs[0,0])
plt.hist(np.squeeze(inde_subs_uncert[:,:,:-2]).flatten(),bins=100,range=(0,0.005))
plt.ylabel('Subs uncert hist\nselect water vapor uncert')
plt.title('Reflectance (all)')

ax = plt.subplot(gs[0,1])
plt.hist(np.squeeze(inde_subs_uncert[:,:,-1]),bins=20,range=(0,0.03))
plt.title('Water Vapor')

ax = plt.subplot(gs[0,2])
plt.hist(np.squeeze(inde_subs_uncert[:,:,-2]),bins=20,range=(0,0.22))
plt.title('AOD')

# Subs uncertainty, low
ax = plt.subplot(gs[0,0])
plt.hist(np.squeeze(inde_subs_uncert[low_vec_idx,:,:-2]).flatten(),bins=100,range=(0,0.005))
plt.ylabel('Subs uncert hist\nselect water vapor uncert')
plt.title('Reflectance (all)')

ax = plt.subplot(gs[0,1])
plt.hist(np.squeeze(inde_subs_uncert[low_vec_idx,:,-1]),bins=20,range=(0,0.03))
plt.title('Water Vapor')

ax = plt.subplot(gs[0,2])
plt.hist(np.squeeze(inde_subs_uncert[low_vec_idx,:,-2]),bins=20,range=(0,0.22))
plt.title('AOD')

# Subs uncertainty, high
ax = plt.subplot(gs[0,0])
plt.hist(np.squeeze(inde_subs_uncert[high_vec_idx,:,:-2]).flatten(),bins=100,range=(0,0.005))

ax = plt.subplot(gs[0,1])
plt.hist(np.squeeze(inde_subs_uncert[high_vec_idx,:,-1]),bins=20,range=(0,0.03))

ax = plt.subplot(gs[0,2])
plt.hist(np.squeeze(inde_subs_uncert[high_vec_idx,:,-2]),bins=20,range=(0,0.22))


# Final uncertainty, all
ax = plt.subplot(gs[1,0])
plt.hist(np.squeeze(inde_uncert_final[:,:,:-2]).flatten(),bins=100,range=(0,0.005))
plt.ylabel('Final uncert hist')

ax = plt.subplot(gs[1,1])
plt.hist(np.squeeze(inde_uncert_final[:,:,-1]).flatten(),bins=100,range=(0,0.03))

ax = plt.subplot(gs[1,2])
plt.hist(np.squeeze(inde_uncert_final[:,:,-2]).flatten(),bins=100,range=(0,0.22))

# Final uncertainty, low
ax = plt.subplot(gs[1,0])
plt.hist(np.squeeze(inde_uncert_final[low_img_idx,:-2]).flatten(),bins=100,range=(0,0.005))

ax = plt.subplot(gs[1,1])
plt.hist(np.squeeze(inde_uncert_final[low_img_idx,-1]).flatten(),bins=100,range=(0,0.03))

ax = plt.subplot(gs[1,2])
plt.hist(np.squeeze(inde_uncert_final[low_img_idx,-2]).flatten(),bins=100,range=(0,0.22))

# Final uncertainty, high
ax = plt.subplot(gs[1,0])
plt.hist(np.squeeze(inde_uncert_final[high_img_idx,:-2]).flatten(),bins=100,range=(0,0.005))

ax = plt.subplot(gs[1,1])
plt.hist(np.squeeze(inde_uncert_final[high_img_idx,-1]).flatten(),bins=100,range=(0,0.03))

ax = plt.subplot(gs[1,2])
plt.hist(np.squeeze(inde_uncert_final[high_img_idx,-2]).flatten(),bins=100,range=(0,0.22))



# Subs error, all
error_mat = (get_segmented_img(inde_subs_state,segments)-true_state)/true_state

ax = plt.subplot(gs[2,0])
plt.hist(np.squeeze(error_mat[:,:,:-2]).flatten(),bins=100,range=(-0.1,0.1))
plt.ylabel('Subs (error)/true')

ax = plt.subplot(gs[2,1])
plt.hist(np.squeeze(error_mat[:,:,-1]).flatten(),bins=100,range=(-0.05,0.05))

ax = plt.subplot(gs[2,2])
plt.hist(np.squeeze(error_mat[:,:,-2]).flatten(),bins=100,range=(-1,1))


# Subs error, low
ax = plt.subplot(gs[2,0])
plt.hist(np.squeeze(error_mat[low_img_idx,:-2]).flatten(),bins=100,range=(-0.1,0.1))
plt.ylabel('Subs (error)/true')

ax = plt.subplot(gs[2,1])
plt.hist(np.squeeze(error_mat[low_img_idx,-1]).flatten(),bins=100,range=(-0.05,0.05))

ax = plt.subplot(gs[2,2])
plt.hist(np.squeeze(error_mat[low_img_idx,-2]).flatten(),bins=100,range=(-1,1))

# Subs error, high
ax = plt.subplot(gs[2,0])
plt.hist(np.squeeze(error_mat[high_img_idx,:-2]).flatten(),bins=100,range=(-0.1,0.1))

ax = plt.subplot(gs[2,1])
plt.hist(np.squeeze(error_mat[high_img_idx,-1]).flatten(),bins=100,range=(-0.05,0.05))

ax = plt.subplot(gs[2,2])
plt.hist(np.squeeze(error_mat[high_img_idx,-2]).flatten(),bins=100,range=(-1,1))



# Final error, all
error_mat = (inde_state-true_state)/true_state

ax = plt.subplot(gs[3,0])
plt.hist(np.squeeze(error_mat[:,:,:-2]).flatten(),bins=100,range=(-0.1,0.1))
plt.ylabel('Final (error)/true')

ax = plt.subplot(gs[3,1])
plt.hist(np.squeeze(error_mat[:,:,-1]).flatten(),bins=100,range=(-0.05,0.05))

ax = plt.subplot(gs[3,2])
plt.hist(np.squeeze(error_mat[:,:,-2]).flatten(),bins=100,range=(-1,1))

# Final error, low
ax = plt.subplot(gs[3,0])
plt.hist(np.squeeze(error_mat[low_img_idx,:-2]).flatten(),bins=100,range=(-0.1,0.1))

ax = plt.subplot(gs[3,1])
plt.hist(np.squeeze(error_mat[low_img_idx,-1]).flatten(),bins=100,range=(-0.05,0.05))

ax = plt.subplot(gs[3,2])
plt.hist(np.squeeze(error_mat[low_img_idx,-2]).flatten(),bins=100,range=(-1,1))

# Final error, high
ax = plt.subplot(gs[3,0])
plt.hist(np.squeeze(error_mat[high_img_idx,:-2]).flatten(),bins=100,range=(-0.1,0.1))

ax = plt.subplot(gs[3,1])
plt.hist(np.squeeze(error_mat[high_img_idx,-1]).flatten(),bins=100,range=(-0.05,0.05))

ax = plt.subplot(gs[3,2])
plt.hist(np.squeeze(error_mat[high_img_idx,-2]).flatten(),bins=100,range=(-1,1))




In [ ]:
error_mat.shape

In [ ]:
inde_state.shape

In [ ]:
plt.close('all')

In [ ]:
percentile_q = 5
low_vec_idx = uncert_base_vec<=np.percentile(uncert_base_vec,q=percentile_q)
low_img_idx = uncert_base_img<=np.percentile(uncert_base_vec,q=percentile_q)
high_vec_idx = uncert_base_vec>=np.percentile(uncert_base_vec,q=(100-percentile_q))
high_img_idx = uncert_base_img>=np.percentile(uncert_base_vec,q=(100-percentile_q))

fig = plt.figure(figsize=(8,10),constrained_layout = True)
gs = fig.add_gridspec(ncols=3, nrows=4) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

# Subs uncertainty, low
ax = plt.subplot(gs[0,0])
plt.hist(np.squeeze(np.mean(inde_subs_uncert[low_vec_idx,:,:-2],axis=2)))
plt.ylabel('Subs uncert hist\nselect water vapor uncert')
plt.title('Reflectance (mean)')

ax = plt.subplot(gs[0,1])
plt.hist(np.squeeze(inde_subs_uncert[low_vec_idx,:,-1]))
plt.title('Water Vapor')

ax = plt.subplot(gs[0,2])
plt.hist(np.squeeze(inde_subs_uncert[low_vec_idx,:,-2]))
plt.title('AOD')

# Subs uncertainty, high
ax = plt.subplot(gs[0,0])
plt.hist(np.squeeze(np.mean(inde_subs_uncert[high_vec_idx,:,:-2],axis=2)))

ax = plt.subplot(gs[0,1])
plt.hist(np.squeeze(inde_subs_uncert[high_vec_idx,:,-1]))

ax = plt.subplot(gs[0,2])
plt.hist(np.squeeze(inde_subs_uncert[high_vec_idx,:,-2]))

# Final uncertainty, low
ax = plt.subplot(gs[1,0])
plt.hist(np.squeeze(np.mean(inde_uncert_final[low_img_idx,:-2],axis=1)).flatten(),bins=100)
plt.ylabel('Final uncert hist')

ax = plt.subplot(gs[1,1])
plt.hist(np.squeeze(inde_uncert_final[low_img_idx,-1]).flatten(),bins=100)

ax = plt.subplot(gs[1,2])
plt.hist(np.squeeze(inde_uncert_final[low_img_idx,-2]).flatten(),bins=100)

# Final uncertainty, high
ax = plt.subplot(gs[1,0])
plt.hist(np.squeeze(np.mean(inde_uncert_final[high_img_idx,:-2],axis=1)).flatten(),bins=100)

ax = plt.subplot(gs[1,1])
plt.hist(np.squeeze(inde_uncert_final[high_img_idx,-1]).flatten(),bins=100)

ax = plt.subplot(gs[1,2])
plt.hist(np.squeeze(inde_uncert_final[high_img_idx,-2]).flatten(),bins=100)



In [ ]:
inde_uncert_final[low_img_idx,:-2].shape

In [ ]:
# Load true state and simulated radiance
print('Loading {}...'.format(data_dir +'{}_{}_{}_simulated_rdn.hdr'.format(instrument_tag,sim_tag,exp_tag)))
synth_data = envi.open(data_dir +'{}_{}_{}_simulated_rdn.hdr'.format(instrument_tag,sim_tag,exp_tag)).open_memmap(interleave='bip')
# Load retrieved values (only classic, for now)
print('Loading {}...'.format(data_dir+ '{}_{}_{}_true_state.hdr'.format(instrument_tag,sim_tag,exp_tag)))
print('Loading {}...'.format(retrieval_dir + 'output/{}_state.hdr'.format(instrument_tag)))
true_state = envi.open(data_dir+ '{}_{}_{}_true_state.hdr'.format(instrument_tag,sim_tag,exp_tag)).open_memmap(interleave='bip')
inde_state = envi.open(retrieval_dir + 'output/{}_state.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
inde_uncert = envi.open(retrieval_dir + 'output/{}_uncert.hdr'.format(instrument_tag)).open_memmap(interleave='bip')

In [ ]:

plot_chn = 100

fig = plt.figure(figsize=(8,8),constrained_layout = True)
gs = fig.add_gridspec(ncols=3, nrows=3)

ax = plt.subplot(gs[0,0])
plt.imshow(inde_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('Classic Retrieval State')
plt.colorbar(ax=ax,shrink=0.5)
clim_0 = plt.gci().get_clim()
plt.title('Reflectance')

ax = plt.subplot(gs[0,1])
plt.imshow(inde_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
clim_1 = plt.gci().get_clim()
plt.title('Water Vapor')

ax = plt.subplot(gs[0,2])
plt.imshow(inde_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
clim_2 = plt.gci().get_clim()
plt.title('AOD')

ax = plt.subplot(gs[1,0])
plt.imshow(inde_uncert[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('Uncertainty')
plt.colorbar(ax=ax,shrink=0.5)
clim_0 = plt.gci().get_clim()

ax = plt.subplot(gs[1,1])
plt.imshow(inde_uncert[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
clim_1 = plt.gci().get_clim()

ax = plt.subplot(gs[1,2])
plt.imshow(inde_uncert[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
clim_2 = plt.gci().get_clim()

ax = plt.subplot(gs[2,0])
plt.imshow(inde_uncert[:,:,plot_chn]/inde_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('Uncertainty/State')
plt.colorbar(ax=ax,shrink=0.5)
clim_0 = plt.gci().get_clim()

ax = plt.subplot(gs[2,1])
plt.imshow(inde_uncert[:,:,-1]/inde_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
clim_1 = plt.gci().get_clim()

ax = plt.subplot(gs[2,2])
plt.imshow(inde_uncert[:,:,-2]/inde_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
clim_2 = plt.gci().get_clim()

In [ ]:
inde_uncert.shape

In [ ]:
inde_state[158,200,100]


In [ ]:
plt_chn = 100

fig = plt.figure(figsize=(8,5),constrained_layout = True)
gs = fig.add_gridspec(ncols=3, nrows=2)

ax = plt.subplot(gs[0,0])
plt.imshow(inde_rfl[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('RFL, Fixed State')
plt.colorbar(ax=ax,shrink=0.5)
clim_0 = plt.gci().get_clim()

ax = plt.subplot(gs[0,1])
plt.imshow(inde_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
clim_1 = plt.gci().get_clim()

ax = plt.subplot(gs[0,2])
plt.imshow(inde_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
clim_2 = plt.gci().get_clim()

ax = plt.subplot(gs[1,0])
plt.imshow(np.reshape(np.append(inde_subs_rfl[:,:,plot_chn],[0,0]),(39,42))); plt.xticks([]); plt.yticks([])
plt.ylabel('subs')
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_0)

ax = plt.subplot(gs[1,1])
plt.imshow(np.reshape(np.append(inde_subs_state[:,:,-1],[0,0]),(39,42))); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_1)

ax = plt.subplot(gs[1,2])
plt.imshow(np.reshape(np.append(inde_subs_state[:,:,-2],[0,0]),(39,42))); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)
plt.clim(clim_2)

In [ ]:
39*42

In [ ]:
inde_subs_state.shape

In [ ]:
# wv_range = [np.min(np.append(true_state[:,:,-1].copy().flatten(),inde_state[:,:,-1].copy().flatten())), np.max(np.append(true_state[:,:,-1].copy().flatten(),inde_state[:,:,-1].copy().flatten()))]
# aod_range = [np.min(np.append(true_state[:,:,-2].copy().flatten(),inde_state[:,:,-2].copy().flatten())), np.max(np.append(true_state[:,:,-2].copy().flatten(),inde_state[:,:,-2].copy().flatten()))]

plot_chn = 100

# for st_lab, st in zip(['True','Independent'], [true_state,inde_state]):
#     print(st_lab, np.min(st[:,:,-1]), np.max(st[:,:,-1]))
    
# for st_lab, st in zip(['True','Independent'], [true_state, inde_state]):
#     print(st_lab, np.min(st[:,:,-2]), np.max(st[:,:,-2]))


fig = plt.figure(figsize=(8,10),constrained_layout = True)
gs = fig.add_gridspec(ncols=3, nrows=5) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

# True State Values

ax = plt.subplot(gs[0,0])
plt.imshow(true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('True')
plt.title('Reflectance')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[0,1])
plt.imshow(true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.title('Water Vapor')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[0,2])
im = plt.imshow(true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.title('AOD')
plt.colorbar(ax=ax,shrink=0.5)

# Independent (Classic) Retrievals

ax = plt.subplot(gs[1,0])
plt.imshow(inde_rfl[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('SCOE')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[1,1])
plt.imshow(inde_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[1,2])
plt.imshow(inde_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

# Error

ax = plt.subplot(gs[2,0])
plt.imshow(inde_rfl[:,:,plot_chn]-true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('SCOE - True')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[2,1])
plt.imshow(inde_state[:,:,-1]-true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[2,2])
plt.imshow(inde_state[:,:,-2]-true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

# Abs(error)/true state

ax = plt.subplot(gs[3,0])
plt.imshow(np.abs(inde_rfl[:,:,plot_chn]-true_state[:,:,plot_chn])/true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('Abs(err)/true')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[3,1])
plt.imshow(np.abs(inde_state[:,:,-1]-true_state[:,:,-1])/true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[3,2])
plt.imshow(np.abs(inde_state[:,:,-2]-true_state[:,:,-2])/true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.colorbar(ax=ax,shrink=0.5)

# RMSE across reflectances

ax = plt.subplot(gs[4,0])
plt.imshow(np.sqrt(np.mean(np.abs(inde_rfl-true_state[:,:,:-2])**2,axis=2))); plt.xticks([]); plt.yticks([])
plt.ylabel('RMSE over channels')
plt.colorbar(ax=ax,shrink=0.5)

#plt.show()

In [ ]:
fig = plt.figure(figsize=(12,10),constrained_layout = True)
gs = fig.add_gridspec(ncols=6, nrows=4) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

# plot_x = np.array([5,10,18,13,15])
# plot_y = np.array([0,5,6,13,15])

plot_x = np.arange(20,396,70)
plot_y = plot_x

ax = plt.subplot(gs[0,0])
plt.imshow(true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('True')
plt.title('Reflectance')
plt.colorbar(ax=ax,shrink=0.5)
#plt.scatter(plot_x,plot_y,marker='o')

ax = plt.subplot(gs[1,0])
plt.imshow(inde_rfl[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('SCOE, CE')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[2,0])
plt.imshow(inde_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.ylabel('SCOE, CE')
plt.title('Water Vapor')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[3,0])
plt.imshow(inde_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.ylabel('SCOE, CE')
plt.title('AOD')
plt.colorbar(ax=ax,shrink=0.5)


for ii in range(plot_x.shape[0]):
    ax = plt.subplot(gs[:2,1:])
    plt.plot(wl_nan,np.squeeze(true_state[plot_y[ii],plot_x[ii],:-2]))
    plt.plot(wl_nan,np.squeeze(inde_rfl[plot_y[ii],plot_x[ii],:]),color='k',linestyle='dashed')
    plt.title('True and retrieved reflectance')
    plt.legend(['True state','Independent retrieval'])
    
    ax = plt.subplot(gs[2:,1:])
    plt.plot(wl_nan,np.squeeze(synth_data[plot_y[ii],plot_x[ii],:]))
    plt.title('Simulated radiance')
    plt.legend(['Simulated'])
    
     
    ax = plt.subplot(gs[0,0])
    plt.scatter(plot_x[ii],plot_y[ii],marker='o')

In [ ]:
plt.close('all')

In [ ]:
plot_chn = 100

# for st_lab, st in zip(['True','Independent'], [true_state,inde_state]):
#     print(st_lab, np.min(st[:,:,-1]), np.max(st[:,:,-1]))
    
# for st_lab, st in zip(['True','Independent'], [true_state, inde_state]):
#     print(st_lab, np.min(st[:,:,-2]), np.max(st[:,:,-2]))


fig = plt.figure(figsize=(8,10),constrained_layout = True)
gs = fig.add_gridspec(ncols=3, nrows=5) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

# True State Values

ax = plt.subplot(gs[0,0])
plt.imshow(true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('True')
plt.title('Reflectance')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[0,1])
plt.imshow(true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.title('Water Vapor')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[0,2])
im = plt.imshow(true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.title('AOD')
plt.colorbar(ax=ax,shrink=0.5)

err_tag_list = ['ce','cosdis','noerr']

scoe_rfl100_all = np.zeros((true_state.shape[0],true_state.shape[1],len(err_tag_list)))
scoe_h2o_all = np.zeros((true_state.shape[0],true_state.shape[1],len(err_tag_list)))
scoe_aod_all = np.zeros((true_state.shape[0],true_state.shape[1],len(err_tag_list)))
scoe_rfl_rmse_all = scoe_rfl100_all.copy()
for ii in range(len(err_tag_list)):
    err_tag = err_tag_list[ii]
    retrieval_dir = base_dir + 'retrievals/scoe_retrievals_{}/{}_{}_{}{}/'.format(err_tag,instrument_tag,sim_tag,exp_tag,retrieval_tag)

    print('Loading {}...'.format(retrieval_dir + 'output/{}_rfl.hdr'.format(instrument_tag)))
    scoe_rfl = envi.open(retrieval_dir + 'output/{}_rfl.hdr'.format(instrument_tag)).open_memmap(interleave='bip')
    scoe_state = envi.open(retrieval_dir + 'output/{}_fixed_state.hdr'.format(instrument_tag)).open_memmap(interleave='bip')

    
    ax = plt.subplot(gs[ii+1,0])
    plt.imshow(scoe_rfl[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
    plt.ylabel('SCOE, {}'.format(err_tag))
    plt.colorbar(ax=ax,shrink=0.5)

    ax = plt.subplot(gs[ii+1,1])
    plt.imshow(scoe_state[:,:,-1]); plt.xticks([]); plt.yticks([])
    plt.colorbar(ax=ax,shrink=0.5)

    ax = plt.subplot(gs[ii+1,2])
    plt.imshow(scoe_state[:,:,-2]); plt.xticks([]); plt.yticks([])
    plt.colorbar(ax=ax,shrink=0.5)
    
    scoe_rfl100_all[:,:,ii] = scoe_rfl[:,:,plot_chn]
    scoe_h2o_all[:,:,ii] = scoe_state[:,:,-1]
    scoe_aod_all[:,:,ii] = scoe_state[:,:,-2]
    scoe_rfl_rmse_all[:,:,ii] = np.sqrt(np.mean((scoe_rfl-true_state[:,:,:-2])**2,axis=2))

In [ ]:
fig = plt.figure(figsize=(12,10),constrained_layout = True)
gs = fig.add_gridspec(ncols=4, nrows=4) #, wspace=0.5, hspace=0.05)#, width_ratios=widths, height_ratios=heights)

# True State Values

ax = plt.subplot(gs[0,0])
plt.imshow(true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
plt.ylabel('True')
plt.title('Reflectance')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[0,1])
plt.imshow(true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
plt.title('Water Vapor')
plt.colorbar(ax=ax,shrink=0.5)

ax = plt.subplot(gs[0,2])
im = plt.imshow(true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
plt.title('AOD')
plt.colorbar(ax=ax,shrink=0.5)

err_tag_list = ['ce','cosdis','noerr']

for ii in range(len(err_tag_list)):

    ax = plt.subplot(gs[ii+1,0])
    plt.imshow((true_state[:,:,plot_chn]-scoe_rfl100_all[:,:,ii])/true_state[:,:,plot_chn]); plt.xticks([]); plt.yticks([])
    plt.ylabel('Fractional Error\nSCOE, {}'.format(err_tag))
    plt.colorbar(ax=ax,shrink=0.5)

    ax = plt.subplot(gs[ii+1,1])
    plt.imshow((true_state[:,:,-1]-scoe_h2o_all[:,:,ii])/true_state[:,:,-1]); plt.xticks([]); plt.yticks([])
    plt.colorbar(ax=ax,shrink=0.5)

    ax = plt.subplot(gs[ii+1,2])
    plt.imshow((true_state[:,:,-2]-scoe_aod_all[:,:,ii])/true_state[:,:,-2]); plt.xticks([]); plt.yticks([])
    plt.colorbar(ax=ax,shrink=0.5)
    
    ax = plt.subplot(gs[ii+1,3])
    plt.imshow(scoe_rfl_rmse_all[:,:,ii]); plt.xticks([]); plt.yticks([])
    plt.colorbar(ax=ax,shrink=0.5)
    if ii==0:
        plt.title('Overall rfl RMSE')
    